# Build the Neural Network
- 神经网络由对数据执行操作的layers/modules组成。`torch.nn` 命名空间提供了你构建自己的神经网络所需的所有基础组件(building blocks)。
- PyTorch 中的每个module都是 `nn.Module` 的子类。
- 神经网络本身就是一个module，它由其他modules（layerd）组成。这种嵌套结构使得轻松构建和管理复杂的架构成为可能。

在接下来的章节中，我们将构建一个神经网络，对 FashionMNIST 数据集中的图像进行分类。

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
我们希望能够在诸如 CUDA、MPS、MTIA 或 XPU 等加速器上训练我们的模型。如果当前的加速器可用，我们就会使用它。否则，我们将使用 CPU。
- 具体来说，CUDA 是 NVIDIA 推出的一种并行计算平台和编程模型，可加速 GPU 计算；
- MPS 是苹果公司的 Metal Performance Shaders，用于在苹果设备 GPU 上进行高性能计算；
- MTIA 是英特尔推出的一种面向 AI 推理的加速器；
- XPU 是一些厂商自定义的通用计算加速芯片的统称。

这句话表明训练模型优先选择可用的这些加速器，若都不可用则退而求其次使用中央处理器（CPU）来训练模型。


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the class
1. 我们通过继承 `nn.Module` 类来定义神经网络，并在`__init__`方法中初始化神经网络的各层。每个 `nn.Module` 的子类都要在 `forward`方法中实现对输入数据的操作。
2. 然后定义了一个名为 `NeuralNetwork` 的类，它继承自`nn.Module`。
- 具体来说，在 **PyTorch** 框架中，`nn.Module` 是所有神经网络模块的基类。当创建自定义神经网络时，通常会创建一个类继承 `nn.Module`。在这个类的`__init__`函数里，会定义神经网络的各个层，比如全连接层、卷积层等。
- 而 `forward` 函数则定义了数据在网络中的前向传播路径，也就是输入数据如何经过各个层的运算得到输出。
    例如，可能在`__init__`里定义一个线性层```self.linear = nn.Linear(10, 5)```，表示输入维度为 10，输出维度为 5 的线性层，然后在 `forward` 里定义```x = self.linear(x)```来实现数据的前向传播。

In [ ]:
class NeuralNetwork(nn.Module):
    # 定义神经网络类，继承自PyTorch的nn.Module基类
    def __init__(self):
        # 类初始化方法
        super().__init__()  # 调用父类nn.Module的初始化方法, 
        self.flatten = nn.Flatten()  # 创建Flatten层，用于将输入图像展平为一维向量
        
        # 创建Sequential容器，按顺序包含多个神经网络层
        self.linear_relu_stack = nn.Sequential(
            # 第一个全连接层，将28×28=784维输入映射到512维隐藏层
            nn.Linear(28*28, 512),
            nn.ReLU(),  # ReLU激活函数，引入非线性特性
            
            # 第二个全连接层，将512维输入映射到512维隐藏层
            nn.Linear(512, 512),
            nn.ReLU(),  # ReLU激活函数
            
            # 第三个全连接层，将512维输入映射到10维输出（对应10个类别）
            nn.Linear(512, 10),
        )

    def forward(self, x):
        # 定义前向传播过程
        x = self.flatten(x)  # 将输入图像展平为一维向量
        logits = self.linear_relu_stack(x)  # 依次通过线性层和激活函数
        return logits  # 返回最终的未归一化预测值（logits）

> “super ()” 是 Python 的一个内置函数，用于调用父类（超类）的方法。它常出现在类的继承体系中，当子类需要复用父类的方法，同时又要添加一些额外的功能时使用。
>
> “全连接层” 指在神经网络中，**该层的每个神经元都与上一层的所有神经元相连接**。这种连接方式使得全连接层能够整合上一层的所有信息，从而学习到数据中的复杂模式。例如在图像识别任务里，经过卷积层和池化层提取特征后，全连接层可以把这些特征组合起来用于图像分类。它在神经网络中常用于将提取到的特征映射到最终的输出空间，如用于分类任务时，将特征映射到不同类别对应的概率值
>
>ReLU 即修正线性单元（Rectified Linear Unit ）激活函数，是一种在人工神经网络中广泛使用的激活函数。它的数学表达式为：f (x) = max (0, x) ，意思是当输入值 x 大于 0 时，输出就是 x 本身；当输入值 x 小于等于 0 时，输出为 0 。
ReLU 激活函数在神经网络中有诸多优点。比如，它能有效解决梯度消失问题，使得网络在训练过程中更容易收敛。在图像识别领域，使用 ReLU 激活函数的卷积神经网络可以更好地对图像特征进行提取和学习。另外，它计算简单，能够加快网络的训练速度。像在 AlexNet 等经典的神经网络模型中就成功应用了 ReLU 激活函数，提升了模型的性能和训练效率

我们创建一个神经网络`NeuralNetwork`的instance，将其移动到指定`device`上，然后打印出它的结构。

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用该模型，我们向其传入the input data。这会执行模型的 `forward` 方法，同时还会进行一些后台操作。切勿直接调用 `model.forward ()` !!!

调用该模型处理输入数据时，会返回一个二维张量。其中，维度 0 对应每个类别的 10 个原始预测值的每个输出，维度 1 对应每个输出的各个单独值。我们通过将其传递给 `nn.Softmax` 模块的一个实例来获得预测概率。
- 举个例子：假设我们有一个三分类的模型（比如识别猫、狗、鸟），输入一张图片后，模型输出的原始预测值是一个包含 3 个元素的数组，例如 [1.2, 0.8, -0.3]。这些数值是模型对每个类别的 “打分”，但它们的范围不确定，也不能直接解释为概率。通过 `Softmax` 函数处理后，这些原始分数会被转换为概率分布，例如 [0.54, 0.38, 0.08]，表示图片属于猫、狗、鸟的概率分别为 54%、38% 和 8%。我们可以直接选择概率最高的类别（这里是猫）作为最终预测结果
- 对于输入向量 $\mathbf{x} = [x_1, x_2, \ldots, x_n]$，Softmax 函数输出的第 i 个元素为：$\text{Softmax}(\mathbf{x})_i = \frac{e^{x_i}}{\sum_{j=1}^n e^{x_j}}$

In [7]:
# 生成一个随机输入张量，模拟一张28×28的单通道图像
# torch.rand(1, 28, 28) 创建一个形状为 [1, 28, 28] 的张量
# device=device 指定张量存储在指定设备（CPU/GPU）上
X = torch.rand(1, 28, 28, device=device)

# 将输入传递给模型，获得原始预测值（logits）
# logits 的形状通常为 [batch_size, num_classes]
logits = model(X)

# 应用Softmax函数将logits转换为概率分布
# dim=1 表示对第二个维度（类别维度）进行Softmax操作
# pred_probab 的形状与logits相同，但每个样本的类别分数之和为1
pred_probab = nn.Softmax(dim=1)(logits)

# 获取概率最高的类别索引
# argmax(1) 表示在第二个维度（类别维度）上取最大值的索引
# y_pred 是一个形状为 [batch_size] 的张量，包含预测的类别标签
y_pred = pred_probab.argmax(1)

# 打印预测结果
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


## Model Layers
让我们剖析 FashionMNIST 模型中的各个层。为了说明这一点，我们将选取一个包含 3 张 28x28 尺寸图像的小批量样本，看看当它在网络中传递时会发生什么。

In [9]:
# 创建一个形状为[3, 28, 28]的随机张量，
# 模拟一个3通道（如RGB）的28×28像素图像
input_image = torch.rand(3, 28, 28)

# 打印张量的维度信息，输出应为torch.Size([3, 28, 28])
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten
We initialize the [`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous（不间断的） array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

The [`linear layer`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU
非线性激活函数构建了模型输入与输出之间的复杂映射关系。它们在线性变换之后应用，用于引入非线性，帮助神经网络学习各种各样的现象。

在这个模型中，我们在线性层之间使用了神经网络中的 [`ReLU` 函数](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)，但还有其他激活函数可用于在模型中引入非线性。


In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2594, -0.3404,  0.0543,  0.0757,  0.5511,  0.3468,  0.0217,  0.0199,
          0.0876, -0.3879,  0.5041,  0.0974, -0.4897,  0.3758, -0.2685, -0.0617,
         -0.0566, -0.9164,  0.2012, -0.3498],
        [ 0.2341, -0.3851, -0.1324, -0.1336,  0.5921,  0.1998,  0.2289, -0.3141,
          0.1073, -0.5461,  0.5249,  0.3040, -0.1568,  0.3868, -0.0042, -0.2018,
         -0.0434, -0.5307,  0.0763, -0.5658],
        [ 0.2474, -0.5597,  0.1563,  0.1173,  0.6961,  0.0620,  0.1685, -0.2289,
          0.0476, -0.3720,  0.7633,  0.1478, -0.4536,  0.3780,  0.1305,  0.0417,
         -0.1380, -0.6709,  0.3664, -0.4156]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2594, 0.0000, 0.0543, 0.0757, 0.5511, 0.3468, 0.0217, 0.0199, 0.0876,
         0.0000, 0.5041, 0.0974, 0.0000, 0.3758, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2012, 0.0000],
        [0.2341, 0.0000, 0.0000, 0.0000, 0.5921, 0.1998, 0.2289, 0.0000, 0.1073,
         0.0000, 0.5249, 0.3040, 0.0000, 0.3868, 0.00

## nn.Sequential
`nn.Sequential` 是一个有序的模块容器。数据会按照定义的相同顺序依次通过所有模块。你可以使用顺序容器，像如下  `seq_modules` 那样快速搭建一个网络。

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.2262, -0.1857,  0.2115,  0.0048, -0.2180, -0.0279,  0.2536,  0.0969,
         -0.2188, -0.4782],
        [-0.1118, -0.0789,  0.0946,  0.0347, -0.2986,  0.2486,  0.2058,  0.2379,
         -0.2071, -0.4147],
        [-0.0880, -0.1769,  0.1025,  0.0639, -0.1503,  0.1200,  0.2279,  0.1921,
         -0.2319, -0.4943]], grad_fn=<AddmmBackward0>)


## nn.Softmax
神经网络的最后一个线性层返回 *logits*（即取值范围为 [-∞, ∞] 的原始值），这些值会被传递到 `nn.Softmax` 模块。*logits* 会被缩放至 [0, 1] 的取值范围，该范围的值代表模型对每个类别的预测概率。`dim` 参数表示沿着该维度，所有值的总和必须为 1


In [20]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0843, 0.0878, 0.1306, 0.1062, 0.0850, 0.1028, 0.1362, 0.1165, 0.0849,
         0.0655],
        [0.0899, 0.0929, 0.1105, 0.1041, 0.0746, 0.1289, 0.1235, 0.1275, 0.0817,
         0.0664],
        [0.0936, 0.0856, 0.1132, 0.1089, 0.0879, 0.1152, 0.1283, 0.1238, 0.0810,
         0.0623]], grad_fn=<SoftmaxBackward0>)


## Model Parameters
在神经网络中，许多层都有参数化设置 (*parameterized*)，也就是说，它们具有相关的*weights* 和 *biases*，这些 *weights* 和 *biases* 在训练过程中会进行优化。对 `nn.Module` 进行子类化会自动跟踪在模型对象内部定义的所有字段，并且可以通过模型的 `parameters()` 或 `named_parameters()` 方法访问所有参数。

在这个例子中，我们遍历每个parameter，并打印其size以及其values的预览。

| **方法**               | **返回值内容**                          | **使用场景**                          | **简单比喻**                          |
|------------------------|---------------------------------------|-----------------------------------|-----------------------------------|
| `model.parameters()`   | 只有参数值，没有名字                    | 当你想一次性处理所有参数时（如“全部参数乘以0.1”）     | 像一个装满零件的箱子，你只关心零件本身，不关心名字  |
| `model.named_parameters()` | 参数值 + 参数名称（如`"layer1.weight"`） | 当你需要区分不同参数时（如“只训练最后一层”）       | 像一个贴了标签的零件箱，每个零件都标着“这是齿轮”“这是螺丝” |

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0223,  0.0267,  0.0308,  ...,  0.0263, -0.0071, -0.0350],
        [-0.0057,  0.0092,  0.0036,  ..., -0.0292,  0.0229,  0.0077]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0225,  0.0322], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0118, -0.0095,  0.0131,  ...,  0.0257, -0.0400,  0.0333],
        [ 0.0264, -0.0146, -0.0189,  ...,  0.0398,  0.0343, -0.0200]],
       device='cuda:0', grad_fn=<Sl